In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, laplace

plt.style.use('ggplot')
x = np.linspace(-10, 10, 1000)
normal_pdf = norm.pdf(x, loc=0, scale=1)
laplace_pdf = laplace.pdf(x, loc=0, scale=1/np.sqrt(2))

plt.figure(figsize=(8, 5))
plt.plot(x, normal_pdf, label='Normal(0, 1)', color='blue', linestyle='-')
plt.plot(x, laplace_pdf, label=r'Laplace$(0, \frac{1}{\sqrt{2}})$', color='orange', linestyle='--')
plt.title('Comparison of Normal and Laplace Distributions')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
def soft_threshold(z, t):
    return np.sign(z) * np.maximum(np.abs(z) - t, 0.0)

def sure_soft_threshold(z, t, sigma=1.0):
    z_hat = soft_threshold(z, t)
    rss = np.sum((z_hat - z) ** 2)
    df = np.sum(np.abs(z) > t)
    return (rss + 2.0 * sigma**2 * df) / len(z)

def find_best_threshold_sure(z, sigma=1.0):
    best_t, best_s = 0.0, float('inf')
    for t in np.unique(np.abs(z)):        
        s = sure_soft_threshold(z, t, sigma)
        if s < best_s:
            best_s, best_t = s, t
    return best_t, best_s

def measure_fdr_fp_tpr(X, thresh, mu_true):
    rejected = np.abs(X) > thresh
    R = rejected.sum()
    fp = ((mu_true == 0) & rejected).sum()
    tp = ((mu_true != 0) & rejected).sum()
    pos = (mu_true != 0).sum()
    fdr = 0.0 if R == 0 else fp / R
    tpr = 0.0 if pos == 0 else tp / pos
    return fdr, fp, tpr

def generate_sparse_means(n, p, sigma, *, rng):
    is_zero = rng.random(n) < p
    mu = np.zeros(n)
    mu[~is_zero] = rng.normal(0.0, sigma, size=(~is_zero).sum())
    return mu

def generate_observations(mu, laplace_b, *, rng):
    noise = rng.laplace(0.0, laplace_b, size=len(mu))
    return mu + noise

def run_single_simulation(n, p, sigma=6.08, lap_b=1/np.sqrt(2), *, rng):
    mu_true = generate_sparse_means(n, p, sigma, rng=rng)
    X = generate_observations(mu_true, lap_b, rng=rng)

    lam_fixed = np.sqrt(2*np.log(n))
    Xhat_fixed = soft_threshold(X, lam_fixed)
    mse_fixed = np.mean((Xhat_fixed - mu_true) ** 2)
    fdr_fixed, fp_fixed, tpr_fixed = measure_fdr_fp_tpr(X, lam_fixed, mu_true)

    lam_sure, _ = find_best_threshold_sure(X, sigma=1.0)
    Xhat_sure = soft_threshold(X, lam_sure)
    mse_sure = np.mean((Xhat_sure - mu_true) ** 2)
    fdr_sure, fp_sure, tpr_sure = measure_fdr_fp_tpr(X, lam_sure, mu_true)
    
    #for table and plotting
    return (mse_fixed, mse_sure,fdr_fixed, fdr_sure, 
            fp_fixed,  fp_sure, tpr_fixed, tpr_sure, lam_fixed, lam_sure)

def run_simulation(n, p, n_repeats=600, sigma=6.08, lap_b=1/np.sqrt(2)):
    rng = np.random.default_rng()
    out = np.zeros((n_repeats, 10))   # 10 metrics now
    for k in range(n_repeats):
        out[k] = run_single_simulation(n, p, sigma, lap_b, rng=rng)
    return out.mean(axis=0)

n = 2800
sigma = 6.08
lap_b = 1 / np.sqrt(2)
n_repeats = 1000
p_values = np.arange(0.90, 0.99, 0.01)

metrics = {k: [] for k in ("mse_fixed", "mse_sure", "fdr_fixed", "fdr_sure", "fp_fixed",  "fp_sure",
            "tpr_fixed", "tpr_sure", "lam_fixed", "lam_sure")}

print(f"{'p':<6}"
      f"{'MSE_fixed':<10}{'MSE_sure':<10}"
      f"{'FDR_fixed':<11}{'FDR_sure':<11}"
      f"{'FP_fixed':<10}{'FP_sure':<10}"
      f"{'TPR_fixed':<11}{'TPR_sure':<11}"
      f"{'lam_fixed':<10}{'lam_sure':<10}")

for p in p_values:
    (mse_f, mse_s,fdr_f, fdr_s,fp_f,  fp_s,tpr_f, tpr_s,lam_f, lam_s) = run_simulation(n, p, n_repeats, sigma, lap_b)
    
    metrics["mse_fixed"].append(mse_f)
    metrics["mse_sure" ].append(mse_s)
    metrics["fdr_fixed"].append(fdr_f)
    metrics["fdr_sure" ].append(fdr_s)
    metrics["fp_fixed" ].append(fp_f)
    metrics["fp_sure" ].append(fp_s)
    metrics["tpr_fixed"].append(tpr_f)
    metrics["tpr_sure" ].append(tpr_s)
    metrics["lam_fixed"].append(lam_f)
    metrics["lam_sure" ].append(lam_s)
    
    #table
    print(f"{p:<6.2f}" f"{mse_f:<10.4f}{mse_s:<10.4f}" f"{fdr_f:<11.4f}{fdr_s:<11.4f}"
          f"{fp_f:<10.4f}{fp_s:<10.4f}" f"{tpr_f:<11.4f}{tpr_s:<11.4f}" f"{lam_f:<10.4f}{lam_s:<10.4f}")

plt.style.use('ggplot')
def make_lineplot(y1, y2, ylabel, title):
    plt.figure(figsize=(6,5))
    plt.plot(p_values, y1, 'o--', label='Fixed Threshold',  color='steelblue')
    plt.plot(p_values, y2, 'o--', label='SURE Threshold',   color='darkorange')
    plt.xlabel('Sparsity (p)')
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()

make_lineplot(metrics["mse_fixed"], metrics["mse_sure"],
              'Average MSE', 'MSE vs. Sparsity (Laplace noise)')
make_lineplot(metrics["fdr_fixed"], metrics["fdr_sure"],
              'Average FDR', 'FDR vs. Sparsity (Laplace noise)')
make_lineplot(metrics["tpr_fixed"], metrics["tpr_sure"],
              'Average TPR', 'TPR vs. Sparsity (Laplace noise)')